# This Notebook presents an expriment of Theory Of Mind on Rock Paper Scissors
### The goal is to research and devlop a method for generic multi-agent games using Theory of Mind modeling.
### We will introduce the tradeoff between choosing optimal move and staying unpredictable.

In [1]:
# Import libraries.
using Turing, StatsPlots, Random

## The model of agent as universal Probalistic Model
### gets some parmeters:
<ul>
    <li> opponent agent - representation of our belief on the opponent model (Turing.jl model)
    <li> my history - list of all moves this player done (At this point unused)
    <li> opponent history - list of all moves the opponent done, used to estimate a prior on the moves
    <li> depth - the depth we want the agent will dive modeling the mind of opponent agent
</ul>        

In [2]:
function best_move(opp_move)
    best_moves = Dict(1 => 2, 2 => 3, 3 => 1)
    move = best_moves[opp_move]
#     println(move)
#     p_deceive = 0.001
#     print(p_deceive)
#     deceive ~ Bernoulli(p_deceive)
#     print(deceive)
#     if deceive
#         move = opp_move
#     end
    return move
end

best_move (generic function with 1 method)

In [3]:
function most_common(samples)
    count = Dict(1 => 0, 2 => 0, 3 => 0)
    for i in 1:length(samples)
        count[samples[i]] += 1
    end
    max_k, max_v = -1 , -1
    for (k, v) in count
        if v > max_v
            max_k , max_v = k, v
        end
    end
    return max_k
end

most_common (generic function with 1 method)

In [4]:
@model function agent(opponent_agent, opponent_history, opp_prior_policy = [1,1,1], depth = 1, discrete_sampler = PG, discrete_sampler_hyper_param=3, num_of_iterations=50)
    if depth == 0
        beta_opp ~ Dirichlet(opp_prior_policy)
        for opp_move in opponent_history
            opp_move ~ Categorical(beta_opp) # observe
        end
        next_opp_move ~ Categorical(beta_opp) # sample
    else
        opponent_model = opponent_agent(agent, opponent_history, opp_prior_policy, depth-1)
        next_opp_move_chain  = sample(opponent_model, IS(), num_of_iterations, progress = false); # if you use MH - you need to burn samples in the chain first
        result = generated_quantities(opponent_model, next_opp_move_chain)
        next_opp_move_from_chain = most_common(result)
        next_opp_move ~ Dirac(next_opp_move_from_chain)
    end
    move = best_move(next_opp_move)
    return move
end

agent (generic function with 6 methods)

the infece is about the opponent model and not my model

### This method is used for the simulation, each model make a move at the end of this function, as a result of reasoning of the model

In [5]:
function move(agent, other_agent, other_agent_history, other_player_prior, my_depth=1)
    other_agent_history = length(other_agent_history) > 0 ? other_agent_history : [1]
    other_agent_history = Array{Int}(other_agent_history)
    m = agent(other_agent, other_agent_history, other_player_prior, my_depth)
    chain = sample(m, IS(), 50, progress = false)
    moves = generated_quantities(m, chain)
    return most_common(moves)
end

move (generic function with 2 methods)

### Game simulation , given two agent models , depth params, let them play num of simulation times

In [6]:
function game(first_player_depth = 1, second_player_depth = 1, first_player_prior_policy = [1,1,1] , second_player_prior_policy = [1, 1, 1])
    first_player = agent
    second_player = agent
#     first_player_prior_policy = [1, 1, 1]
#     second_player_prior_policy = [1, 1, 1]
    num_of_simulations = 100
    first_player_history = []
    second_player_history = []
    for i in 1:num_of_simulations
        m1 = move(first_player, second_player, second_player_history, second_player_prior_policy, first_player_depth)
        push!(first_player_history, m1)
        m2 = move(second_player, first_player, first_player_history, first_player_prior_policy, second_player_depth)
        push!(second_player_history, m2)
    end
    return first_player_history, second_player_history
 end

game (generic function with 5 methods)

In [7]:
function score(history)
    first_player_history, second_player_history = history
    first_wins = 0
    ties = 0
    second_wins = 0
    wins = Dict(1 => 3, 2 => 1, 3 => 2)
    for i in 1:length(first_player_history)
        if wins[first_player_history[i]] == second_player_history[i]
            second_wins += 1
        elseif wins[second_player_history[i]] == first_player_history[i]
            first_wins += 1
        else
            ties += 1
        end
    end
    return first_wins, ties, second_wins
end

score (generic function with 1 method)

In [9]:
score([[1, 2], [3, 1]])

(0, 0, 2)

In [8]:
function display_score(score)
    num_of_wins_first, num_of_ties, num_of_wins_second = score
    println("first player won: $num_of_wins_first")
    println("second player won: $num_of_wins_second") 
    println("ties: $num_of_ties") 
end

display_score (generic function with 1 method)

both player plays without theory of mind - equivilent

In [11]:
display_score(score(game(0, 0)))

first player won: 27
second player won: 32
ties: 41


2nd player plays with one level of theory of mind -> 2nd wins

In [9]:
display_score(score(game(0, 1)))

first player won: 38
second player won: 35
ties: 27


In [12]:
display_score(score(game(0, 1)))

first player won: 32
second player won: 27
ties: 41


1st player plays with one level of theory of mind -> 1st wins

In [10]:
display_score(score(game(1, 0)))

first player won: 40
second player won: 31
ties: 29


In [13]:
display_score(score(game(1, 0)))

first player won: 33
second player won: 28
ties: 39


both players play with one level of theory of mind -> preety much equivelent

In [11]:
display_score(score(game(1, 1)))

first player won: 31
second player won: 33
ties: 36


In [14]:
display_score(score(game(1, 1)))

first player won: 32
second player won: 28
ties: 40


In [12]:
display_score(score(game(0, 0, [20,1,1],[1,1,1])))

first player won: 37
second player won: 35
ties: 28


In [17]:
display_score(score(game(0, 0, [20,1,1],[1,1,1])))

first player won: 30
second player won: 36
ties: 34


In [13]:
display_score(score(game(0, 1, [20,1,1],[1,1,1])))

first player won: 43
second player won: 28
ties: 29


In [18]:
display_score(score(game(0, 1, [20,1,1],[1,1,1])))

first player won: 38
second player won: 35
ties: 27


In [19]:
display_score(score(game(1, 0, [20,1,1],[1,1,1])))

first player won: 28
second player won: 46
ties: 26


In [14]:
display_score(score(game(1, 0, [20,1,1],[1,1,1])))

first player won: 27
second player won: 55
ties: 18


In [20]:
display_score(score(game(1, 1, [20,1,1],[1,1,1])))

first player won: 12
second player won: 33
ties: 55


In [15]:
display_score(score(game(1, 1, [20,1,1],[1,1,1])))

first player won: 18
second player won: 28
ties: 54


In [21]:
display_score(score(game(0, 0,[1,1,1], [20,1,1])))

first player won: 28
second player won: 34
ties: 38


In [16]:
display_score(score(game(0, 0,[1,1,1], [20,1,1])))

first player won: 39
second player won: 31
ties: 30


In [22]:
display_score(score(game(0, 1,[1,1,1], [20,1,1])))

first player won: 58
second player won: 28
ties: 14


In [18]:
display_score(score(game(0, 1,[1,1,1], [20,1,1])))

first player won: 54
second player won: 31
ties: 15


In [23]:
display_score(score(game(1, 0,[1,1,1], [20,1,1])))

first player won: 36
second player won: 41
ties: 23


In [17]:
display_score(score(game(1, 0,[1,1,1], [20,1,1])))

first player won: 30
second player won: 40
ties: 30


In [24]:
display_score(score(game(1, 1,[1,1,1], [20,1,1])))

first player won: 24
second player won: 17
ties: 59


In [19]:
display_score(score(game(1, 1,[1,1,1], [20,1,1])))

first player won: 24
second player won: 21
ties: 55


In [25]:
display_score(score(game(0, 2)))

first player won: 38
second player won: 28
ties: 34


In [20]:
display_score(score(game(0, 2)))

first player won: 34
second player won: 37
ties: 29


In [26]:
display_score(score(game(2, 0)))

first player won: 41
second player won: 29
ties: 30


In [21]:
display_score(score(game(2, 0)))

first player won: 41
second player won: 34
ties: 25


In [27]:
display_score(score(game(1, 2)))

first player won: 51
second player won: 16
ties: 33


In [9]:
display_score(score(game(1, 2)))

first player won: 50
second player won: 24
ties: 26


In [28]:
display_score(score(game(2, 1)))

first player won: 20
second player won: 57
ties: 23


In [10]:
display_score(score(game(2, 1)))

first player won: 18
second player won: 54
ties: 28


In [11]:
display_score(score(game(0, 2,[1,1,1], [20,1,1])))

first player won: 0
second player won: 98
ties: 2


In [12]:
display_score(score(game(0, 2,[20,1,1], [1,1,1])))

first player won: 36
second player won: 34
ties: 30


In [ ]:
display_score(score(game(2, 0,[1,1,1], [20,1,1])))

In [ ]:
display_score(score(game(2, 0, [20,1,1],[1,1,1])))

In [ ]:
display_score(score(game(1, 2, [20,1,1] ,[1,1,1])))

In [ ]:
display_score(score(game(1, 2,[1,1,1], [20,1,1])))

In [ ]:
display_score(score(game(2, 1, [20,1,1] ,[1,1,1])))

In [ ]:
display_score(score(game(2, 1 ,[1,1,1], [20,1,1])))

In [ ]:
display_score(score(game(2, 2, [20,1,1] ,[1,1,1])))

In [ ]:
display_score(score(game(2, 2, [1,1,1], [20,1,1])))

In [ ]:
display_score(score(game(2, 2)))

In [ ]:
display_score(score(game(0, 3)))

In [ ]:
display_score(score(game(3, 0)))